## Finding Hubs and Zones in Pricing Data

Figuring out how to find Hubs or Zones and then query those location names are common first tasks when working with LMP data. This notebook shows some basic queries to help users of the gridstatusio API determine the correct location_type filters to use in order to get the correct locations.

In [13]:
from gridstatusio import GridStatusClient

client = GridStatusClient()

## SPP

Two hubs in the lmp data. The by_bus dataset does not include hubs.

In [2]:
df_spp = client.get_dataset(
    dataset="spp_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Hub",
)
df_spp

Fetching Page 1...Done in 0.2 seconds. 

Total number of rows: 2


,interval_start_utc,interval_end_utc,market,location,location_type,pnode,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,SPPNORTH_HUB,Hub,SPPNORTH_H,12.4283,11.2583,1.2922,-0.1222
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,SPPSOUTH_HUB,Hub,SPPSOUTH_H,-8.7672,11.2584,-19.2451,-0.7805


## MISO
MISO labels many nodes as "Hub" in the raw data, but there are a much smaller subset that are actually state-level Hubs which people look for. To get down to those we filter on the `location_type` of "Hub" and then further filter the dataset based on location names.

In [3]:
df_miso = client.get_dataset(
    dataset="miso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Hub",
)

df_miso_hubs = df_miso[df_miso["location"].str.contains("HUB")]
df_miso_hubs

Fetching Page 1...Done in 0.19 seconds. 

Total number of rows: 406


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
83,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,ARKANSAS.HUB,Hub,19.86,19.09,1.56,-0.79
264,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,ILLINOIS.HUB,Hub,9.64,19.09,-9.11,-0.34
265,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,INDIANA.HUB,Hub,27.19,19.09,7.19,0.91
279,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,LOUISIANA.HUB,Hub,21.19,19.09,2.02,0.08
310,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MICHIGAN.HUB,Hub,26.60,19.09,6.10,1.41
311,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MINN.HUB,Hub,12.15,19.09,-6.45,-0.49
319,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MS.HUB,Hub,21.00,19.09,2.13,-0.22
388,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TEXAS.HUB,Hub,20.49,19.09,1.70,-0.30


## ERCOT
ERCOT has 5 Hubs and also publishes two averages, one each for Buses and Hubs.

In [4]:
df_ercot_hubs = client.get_dataset(
    dataset="ercot_spp_real_time_15_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:15",
    filter_column="location_type",
    filter_value="Trading Hub",
)
df_ercot_hubs

Fetching Page 1...Done in 0.24 seconds. 

Total number of rows: 7


,interval_start_utc,interval_end_utc,location,location_type,market,spp
0,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_BUSAVG,Trading Hub,REAL_TIME_15_MIN,19.96
1,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_HOUSTON,Trading Hub,REAL_TIME_15_MIN,20.30
2,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_HUBAVG,Trading Hub,REAL_TIME_15_MIN,20.13
3,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_NORTH,Trading Hub,REAL_TIME_15_MIN,19.86
4,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_PAN,Trading Hub,REAL_TIME_15_MIN,11.57
5,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_SOUTH,Trading Hub,REAL_TIME_15_MIN,19.39
6,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_WEST,Trading Hub,REAL_TIME_15_MIN,20.98


ERCOT also publishes data for several load zones

In [5]:
df_ercot_zones = client.get_dataset(
    dataset="ercot_spp_real_time_15_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:15",
    filter_column="location_type",
    filter_value="Load Zone",
)
df_ercot_zones

Fetching Page 1...Done in 0.18 seconds. 

Total number of rows: 8


,interval_start_utc,interval_end_utc,location,location_type,market,spp
0,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_AEN,Load Zone,REAL_TIME_15_MIN,20.86
1,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_CPS,Load Zone,REAL_TIME_15_MIN,21.31
2,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_HOUSTON,Load Zone,REAL_TIME_15_MIN,20.31
3,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_LCRA,Load Zone,REAL_TIME_15_MIN,20.49
4,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_NORTH,Load Zone,REAL_TIME_15_MIN,19.83
5,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_RAYBN,Load Zone,REAL_TIME_15_MIN,19.72
6,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_SOUTH,Load Zone,REAL_TIME_15_MIN,23.03
7,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_WEST,Load Zone,REAL_TIME_15_MIN,22.93


## CAISO
CAISO has 3 Trading Hubs in their LMP data.

In [6]:
df_caiso = client.get_dataset(
    dataset="caiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Trading Hub",
)
df_caiso

Fetching Page 1...Done in 0.31 seconds. 

Total number of rows: 3


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TH_NP15_GEN-APND,Trading Hub,-3.99571,-3.29997,-0.70498,0.00924
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TH_SP15_GEN-APND,Trading Hub,-3.16203,-3.29997,0.00000,0.13794
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TH_ZP26_GEN-APND,Trading Hub,-3.14685,-3.29997,0.00000,0.15312


CAISO also publishes data for several DLAPs (Default Load Aggregation Points) which is the node at which all bids for demand are submitted and settled

In [7]:
df_caiso_dlap = client.get_dataset(
    dataset="caiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="DLAP",
)
df_caiso_dlap

Fetching Page 1...Done in 0.35 seconds. 

Total number of rows: 6


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_PACE_NPM-APND,DLAP,-3.29844,-3.29997,0.00120,0.00033
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_PACW_NPM-APND,DLAP,-3.29745,-3.29997,0.00120,0.00132
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_PGAE-APND,DLAP,-4.27954,-3.29997,-0.93997,-0.03960
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_SCE-APND,DLAP,-3.23694,-3.29997,0.00000,0.06303
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_SDGE-APND,DLAP,-3.24420,-3.29997,0.00000,0.05577
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_VEA-APND,DLAP,-3.09768,-3.29997,0.00000,0.20229


## PJM Hubs
As the largest market in the US, PJM also has a higher number of hubs than most markets.

In [8]:
df_pjm = client.get_dataset(
    dataset="pjm_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="HUB",
)
df_pjm

Fetching Page 1...Done in 0.21 seconds. 

Total number of rows: 12


,interval_start_utc,interval_end_utc,market,location,location_id,location_short_name,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,AEP-DAYTON HUB,34497127,AEP-DAYTON HUB,HUB,31.68,31.26,0.41,0.01
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,AEP GEN HUB,34497125,AEP GEN HUB,HUB,31.45,31.26,0.75,-0.56
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,ATSI GEN HUB,116013751,ATSI GEN HUB,HUB,31.46,31.26,0.46,-0.26
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CHICAGO GEN HUB,33092311,CHICAGO GEN HUB,HUB,24.69,31.26,-5.18,-1.39
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CHICAGO HUB,33092313,CHICAGO HUB,HUB,25.32,31.26,-4.94,-1.00
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DOMINION HUB,35010337,DOMINION HUB,HUB,31.95,31.26,0.74,-0.05
6,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,EASTERN HUB,51217,EASTERN HUB,HUB,32.14,31.26,0.62,0.26
7,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,NEW JERSEY HUB,4669664,NEW JERSEY HUB,HUB,31.89,31.26,0.60,0.03
8,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,N ILLINOIS HUB,33092315,N ILLINOIS HUB,HUB,25.11,31.26,-5.01,-1.14
9,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,OHIO HUB,34497151,OHIO HUB,HUB,31.63,31.26,0.30,0.07


## NYISO
NYISO doesn't calculate and publish hubs as a separate category. The market doesn't price at every bus, only at generators and the zonal level, so we will get the Zone names instead. This list includes external regions such as Ontario (OH) and Quebec (HQ).

In [9]:
df_nyiso = client.get_dataset(
    dataset="nyiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Zone",
)
df_nyiso

Fetching Page 1...Done in 0.19 seconds. 

Total number of rows: 15


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CAPITL,Zone,20.13,19.58,-0.00,0.55
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CENTRL,Zone,19.60,19.58,-0.00,0.02
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DUNWOD,Zone,20.56,19.58,-0.00,0.98
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,GENESE,Zone,19.31,19.58,-0.00,-0.27
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,H Q,Zone,19.58,19.58,-0.00,0.00
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,HUD VL,Zone,20.48,19.58,-0.00,0.90
6,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,LONGIL,Zone,20.66,19.58,-0.00,1.08
7,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MHK VL,Zone,20.15,19.58,-0.00,0.57
8,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MILLWD,Zone,20.50,19.58,-0.00,0.92
9,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,NORTH,Zone,19.62,19.58,-0.00,0.04


## ISO-NE
ISO-NE only has a single internal Hub, so we will also query the Load Zones.

In [10]:
df_isone = client.get_dataset(
    dataset="isone_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="HUB",
)
df_isone

Fetching Page 1...Done in 0.17 seconds. 

Total number of rows: 1


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.H.INTERNAL_HUB,HUB,34.91,34.69,0.05,0.17


In [11]:
df_isone_load_zone = client.get_dataset(
    dataset="isone_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="LOAD ZONE",
)
df_isone_load_zone

Fetching Page 1...Done in 0.36 seconds. 

Total number of rows: 8


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.CONNECTICUT,LOAD ZONE,34.01,34.69,0.05,-0.73
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.MAINE,LOAD ZONE,34.01,34.69,-0.39,-0.29
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.NEMASSBOST,LOAD ZONE,35.27,34.69,0.05,0.53
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.NEWHAMPSHIRE,LOAD ZONE,34.98,34.69,0.05,0.24
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.RHODEISLAND,LOAD ZONE,34.81,34.69,0.05,0.07
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.SEMASS,LOAD ZONE,35.20,34.69,0.05,0.46
6,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.VERMONT,LOAD ZONE,34.63,34.69,0.05,-0.11
7,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.WCMASS,LOAD ZONE,34.84,34.69,0.05,0.10


Now let's print out the lists of locations from each market.

In [12]:
print("\nCAISO Trading Hub:", list(df_caiso["location"].unique()))
print("CAISO DLAP:", list(df_caiso_dlap["location"].unique()), "\n")
print("ERCOT Trading Hub:", list(df_ercot_hubs["location"].unique()))
print("ERCOT Load Zone:", list(df_ercot_zones["location"].unique()), "\n")
print("SPP:", list(df_spp["location"].unique()), "\n")
print("MISO:", list(df_miso_hubs["location"].unique()), "\n")
print("PJM:", list(df_pjm["location"].unique()), "\n")
print("NYISO:", list(df_nyiso["location"].unique()), "\n")
print("ISONE Hub:", list(df_isone["location"].unique()))
print("ISONE Load Zone:", list(df_isone_load_zone["location"].unique()), "\n")


CAISO Trading Hub: ['TH_NP15_GEN-APND', 'TH_SP15_GEN-APND', 'TH_ZP26_GEN-APND']
CAISO DLAP: ['DLAP_PACE_NPM-APND', 'DLAP_PACW_NPM-APND', 'DLAP_PGAE-APND', 'DLAP_SCE-APND', 'DLAP_SDGE-APND', 'DLAP_VEA-APND'] 

ERCOT Trading Hub: ['HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_PAN', 'HB_SOUTH', 'HB_WEST']
ERCOT Load Zone: ['LZ_AEN', 'LZ_CPS', 'LZ_HOUSTON', 'LZ_LCRA', 'LZ_NORTH', 'LZ_RAYBN', 'LZ_SOUTH', 'LZ_WEST'] 

SPP: ['SPPNORTH_HUB', 'SPPSOUTH_HUB'] 

MISO: ['ARKANSAS.HUB', 'ILLINOIS.HUB', 'INDIANA.HUB', 'LOUISIANA.HUB', 'MICHIGAN.HUB', 'MINN.HUB', 'MS.HUB', 'TEXAS.HUB'] 

PJM: ['AEP-DAYTON HUB', 'AEP GEN HUB', 'ATSI GEN HUB', 'CHICAGO GEN HUB', 'CHICAGO HUB', 'DOMINION HUB', 'EASTERN HUB', 'NEW JERSEY HUB', 'N ILLINOIS HUB', 'OHIO HUB', 'WESTERN HUB', 'WEST INT HUB'] 

NYISO: ['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'H Q', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'NORTH', 'NPX', 'N.Y.C.', 'O H', 'PJM', 'WEST'] 

ISONE Hub: ['.H.INTERNAL_HUB']
ISONE Load Zone: ['.Z.CONNECTICU